In [1]:
# This script reads in a text and tries to identify the passages containing most smell terms
# 10 sentences before and 10 sentences after the 'smelly passage' are also selected 

# Marieke.van.Erp@dh.huc.knaw.nl
# 7 July 2021

import re 
import spacy 
nlp = spacy.load("nl_core_news_sm") 
import glob 
import collections

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


/usr/local/lib/python3.9/site-packages/spacy/util.py:730: UserWarning: [W095] Model 'nl_core_news_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
#!python3 -m spacy download nl_core_web_sm


✘ No compatible package found for 'nl_core_web_sm' (spaCy v3.1.0)



In [2]:
# Borrowed from: https://github.com/Odeuropa/findSmellWords/blob/main/findSmellWords-generic.py
def LoadRegexes(fn):
    """
    This function reads the plain-text file 'fn' 
    and assumes that each line is a regular expression.
    It returns a corresponding list of Python's regex objects.
    """
    f = open(fn, "rt")
    res = []
    for s in f:
        s = s.strip()
        if not s: continue
        # s = s.replace("@", "(.?)").replace("*", ".*")
        #print(s)
        res.append(re.compile(s))
    f.close()
    #print(res)
    print("%d regexes loaded from \"%s\"." % (len(res), fn))
    return res

fnRegexes = "dutch_smell_regexp.txt"
smellWordRegexes = LoadRegexes(fnRegexes)

72 regexes loaded from "dutch_smell_regexp.txt".


In [3]:
# Borrowed from: https://github.com/Odeuropa/findSmellWords/blob/main/findSmellWords-generic.py
smellWordCache = {}
def IsSmellWord(s):
    """
    Tests if 's' matches any of the regular expressions in 'smellWordRegexes'.
    """
    global smellWordCache
    if s in smellWordCache: return smellWordCache[s]
    result = False
    for re in smellWordRegexes:
        if re.match(s): result = True; break
    smellWordCache[s] = result
    return result

In [4]:
# Adjustments in line 5 and line 80 for the input documents and where you want to write the tracking too. 

# Make sure the texts aren't too long: for x in *txt ; do split -l 1000 $x $x ; done 

for name in glob.glob("/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/*"):
#for name in glob.glob("test.txt"):
    print(name)
    name_elems = name.split('/')
    outputfile = name_elems[-1] + "_potentially_smelly_passages.txt"
    #outputfile = name + "_potentially_smelly_passages.txt"
    # Open a text file and turn it into a spaCy document so you can split it on a sentence by sentence basis.  
    with open(name, "r", encoding='utf8', errors='ignore') as f:
        text = f.read()  # read all lines into a string
    doc = nlp(text)
    print("loaded input document" + name_elems[5])
    # initialise a score per sentence tracker 
    smell_score_per_sentence = {}
    smell_terms_per_sentence = {}
    sentence_spans = list(doc.sents)
    for (i, item) in enumerate(sentence_spans, start=0):
        nSmellWords = 0
        for token in item:
            isSmellWord = IsSmellWord(token.text)
            if "toevlucht" in token.text:   # ugly hack but the regexp doesn't seem to work here, very strange 
                continue
            if "ontvlucht" in token.text:   # ugly hack but the regexp doesn't seem to work here, very strange 
                continue
            if isSmellWord: 
                nSmellWords += 1
                if i in smell_terms_per_sentence:
                    smell_terms_per_sentence[i].append(token.text)
                else:
                    smell_terms_per_sentence[i] = []
                    smell_terms_per_sentence[i].append(token.text)
                #print(i, token, item)
        smell_score_per_sentence[i] =  nSmellWords
    # clean up the dictionary to only consider sentences with smell term matches    
    for key, value in smell_score_per_sentence.items():
        if value == 0:
            del smell_score_per_sentence[key]
            break
    # this bit should filter out overlapping passages -> it checks if the matching sentences are at least 5 sentences apart 
    # OK, abandoned because I couldn't get this working 
    #od = collections.OrderedDict(sorted(smell_score_per_sentence.items()))  
    #prev = 0
    #smell_score_per_sentence_keys_to_keep = {}
    #print(len(od),len(smell_score_per_sentence),len(smell_score_per_sentence_keys_to_keep))
    #for x in od:
    #    print(x)
        
    #for k, v in od.items():
    #    score = k-prev
       # print(k,prev,score)
    #    if prev == 0:
    #        prev = k
    #    if k-prev > 5 and k>5:
    #        smell_score_per_sentence_keys_to_keep = {k : v}
    #    prev = k
    #smell_score_per_sentence = smell_score_per_sentence_keys_to_keep
    print("done matching")
    # sort the smelliest passage in descending order 
    a1_sorted_keys = sorted(smell_score_per_sentence, key=smell_score_per_sentence.get, reverse=True)
    for r in a1_sorted_keys:
        if smell_score_per_sentence[r] > 0:
            #print(sentence_spans[r])
            for x in range(r-10,r):
                try:
                    print(sentence_spans[x], file=open(outputfile, "a"))
                except:
                    pass 
                #print(sentence_spans[x], "\n", file=open("output.txt", "a"))
            for x in range(r,r+10):
                try:
                    print(sentence_spans[x], file=open(outputfile, "a"))
                except:
                    pass 
            print("\n\n=======\n\n", file=open(outputfile, "a"))
    for r in a1_sorted_keys:
        if smell_score_per_sentence[r] > 0:
            print(name_elems[-1], "\t", r, "\t", smell_score_per_sentence[r], "\t", " ".join(smell_terms_per_sentence[r]), file=open("tracker_NL_20_Sept.txt", "a"))

/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtds
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtbj
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtae
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtbm
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtdt
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtab
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_

loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txteq
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtch
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtco
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtev
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtel
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtcu
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021

loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtbh
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtag
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtbo
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtdv
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtcb
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021-09-14_1942/textsplits/oudennieuwoostin01vale_djvu.txtft
loaded input documenttextsplits
done matching
/Users/marieke/Downloads/wetransfer_teksten_2021

In [8]:
# adjust lines 5 and 15 for the right filenames, yeah very sloppy, but I'm invoking the CRAPL License: https://matt.might.net/articles/crapl/ !  
import glob

counter = 1 
for name in glob.glob("/Users/marieke/Downloads/OdeuropaBenchmarkTextPrep/oudennieuwoostin01vale*_potentially_smelly_passages.txt"):
    name_elems = name.split("/")
    print(name_elems[-1][0:-4])
    with open(name, "r") as f:
        text = f.readlines()
    excerpt = ""
    for x in text:
        if "=======" in x:
        #    print(x)
        #
            fragmentname = "NL_TRAVEL_"+name_elems[-1][0:-4]+"_"+str(counter)+'.txt'
            counter += 1
            print(excerpt, file=open(fragmentname, "a"))
            excerpt = ""
        else:
            excerpt = excerpt + x 
   

oudennieuwoostin01vale_djvu.txtaa_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtbb_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtdf_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtaz_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtew_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtai_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtch_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtel_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtcv_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtdk_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtdp_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtcm_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtfj_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtaw_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txtcp_potentially_smelly_passages
oudennieuwoostin01vale_djvu.txteo_potentially_smelly_passages
oudennie